In [ ]:
import tensorflow as tf
import numpy as np

**Dataset setup**

In [ ]:
trainingData=np.load('./anomalyDetection_Normal.npy')
testData=np.load('./data/anomalyDetection_Issue.npy')

#windows of 50 with stride 1
windowSize=50
trainingData=np.array([trainingData[i:i+windowSize,:] for i in range(0, len(trainingData)-windowSize)])
testData=np.array([testData[i:i+windowSize,:] for i in range(0, len(testData)-windowSize)])

**Define the model**

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, rate=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(embed_dim*4, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)
    
transformerModel=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(windowSize, 85)),
    TransformerBlock(85, 2),
    tf.keras.layers.MaxPool1D(pool_size=2),
    TransformerBlock(85, 2),
    tf.keras.layers.UpSampling1D(size=2),
    TransformerBlock(85, 2)
])

**Training**

In [ ]:
transformerModel.compile(loss='mse',optimizer='adam')
transformerModel.fit(trainingData,trainingData,epochs=100,validation_data=(testData,testData))

In [ ]:
transformerModel(testData)[:,25,:]
import plotly.express as px
px.line(tf.reduce_mean(transformerModel(testData)[:,25,:],axis=-1),facet_col='variable',facet_col_wrap=5,facet_row_spacing=0.01,height=1024)